In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import cnn               # CNN module
import pickle
from time import time

In [2]:
nn=cnn.conv_net()
w0,b0=nn.init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=128)

In [109]:
inp=np.random.randn(128,64,64,32)

In [4]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):     #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    num_ker=kernels.shape[3]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix       # TO-DO: Cache up these calculations
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
    kern = kernels.reshape(-1,num_ker)
    for img in padded:      #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, ind), kern))
        out=(out+biases).reshape(out_row,out_col,num_ker)
        output.append(out)
#     output=np.array([(np.dot(np.take(i,ind),kern)+biases) for i in padded]).reshape(batches,out_row,out_col,num_ker)
#     bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()		#for batches
#     output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+biases).reshape(batches,out_row,out_col,num_ker)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [49]:
%timeit conv0=conv2d(1,inp,w0,b0)

1.93 s ± 246 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit conv0=nn.conv2d(inp,w0,b0)

1.79 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [146]:
conv0=conv2d(1,inp,w0,b0)

In [54]:
%timeit d_X_inp,d_w0,d_b0=nn.conv2d_back(errors=conv0,inp=inp,kernels=w0,biases=b0)

6.34 s ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [147]:
conv0.shape

(128, 64, 64, 128)

In [7]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col))
padded[:,:,padding:-padding,padding:-padding]=ipp

In [8]:
img=padded[0]

In [9]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [10]:
padded.shape

(128, 32, 66, 66)

In [11]:
img.shape

(32, 66, 66)

In [16]:
kern.shape

(288, 128)

In [17]:
%timeit bind=np.arange(batches)[:,None]*d*row*col+ind.ravel()

185 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
bind.shape

(128, 1179648)

In [19]:
ind.shape

(4096, 288)

In [20]:
%%timeit
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(np.take(img, ind), kern)+b0
ans=output.reshape(batches,out_row,out_col,num_ker)

1.37 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).reshape(batches,out_row,out_col,num_ker)

1.7 s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit (np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+b0).reshape(batches,out_row,out_col,num_ker)

1.72 s ± 269 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.take(img,ind).shape,kern.shape

((4096, 288), (288, 128))

In [24]:
(np.dot(np.take(img,ind), kern)).shape

(4096, 128)

In [25]:
np.take(padded, bind).reshape(-1,d*ksz*ksz).shape, kern.shape

((524288, 288), (288, 128))

In [26]:
ans=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+b0).reshape(batches,out_row,out_col,num_ker)

In [27]:
ans.shape

(128, 64, 64, 128)

In [28]:
np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).reshape(batches,out_row,out_col,num_ker).__array_interface__

{'data': (139801886789648, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (128, 64, 64, 128),
 'version': 3}

In [29]:
np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).__array_interface__

{'data': (139801886789648, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (128, 4096, 128),
 'version': 3}

In [7]:
def func_int(a,b,c,d):
    return a+b+c+d
def func(a,b,c,d):
    return func_int,a,b,c,d

In [8]:
x=func(1,2,3,4)

In [9]:
x[0](*x[1:])

10

In [10]:
x

(<function __main__.func_int(a, b, c, d)>, 1, 2, 3, 4)

In [148]:
def max_pool(self,inp,ksize=[2,2],stride=[2,2]):
    #inp[batches,row,col,d], kernels[ksz,ksz], stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    ksz=ksize[0]
    out_row,out_col=((inp.shape[2]-ksz)//stride[0]+1),((inp.shape[3]-ksz)//stride[1]+1)
    batches,d,row,col=inp.shape
    output=[]
    max_index=[]
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(0,row,stride[0])[:,None]
    window=window.ravel()+np.arange(0,col,stride[1])[:,None]*col
    slider=np.arange(d)[:,None]*row*col
    ind=(window.ravel()+slider.ravel()[:,None]).reshape(-1,ksz*ksz)
    for img in inp:         #img[d,row,col]
        x_col=np.take(img, ind)
        m_ind=x_col.argmax(axis=1)
        out=x_col[range(m_ind.size),m_ind].reshape(-1,out_row,out_col)  #out[d,or,oc]
        max_ind=np.take(ind,np.arange(x_col.shape[0])*x_col.shape[1]+m_ind)
        output.append(out)
        max_index.append(max_ind)
    return np.array(output).transpose(0,2,3,1), max_index

In [209]:
%timeit a,b=max_pool(1,conv0)

533 ms ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [210]:
%timeit a,b=nn.max_pool(conv0)

3.38 s ± 8.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [149]:
ksize=[2,2]
stride=[2,2]
ipp=conv0.transpose(0,3,1,2)  #ipp[batches,d,row,col]
ksz=ksize[0]
out_row,out_col=((ipp.shape[2]-ksz)//stride[0]+1),((ipp.shape[3]-ksz)//stride[1]+1)
batches,d,row,col=ipp.shape

In [151]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(0,row,stride[0])[:,None]
window=window.ravel()+np.arange(0,col,stride[1])[:,None]*col
slider=np.arange(d)[:,None]*row*col
ind=(window.ravel()+slider.ravel()[:,None]).reshape(-1,ksz*ksz)

In [152]:
# %%timeit
output=[]
max_index=[]
for img in ipp:         #img[d,row,col]
    x_col=np.take(img, ind)
    m_ind=x_col.argmax(axis=1)
    out=x_col[range(m_ind.size),m_ind].reshape(-1,out_row,out_col)  #out[d,or,oc]
    max_ind=np.take(ind,np.arange(x_col.shape[0])*x_col.shape[1]+m_ind)
    output.append(out)
    max_index.append(max_ind)
a,b=np.array(output).transpose(0,2,3,1), max_index

In [153]:
a.shape,np.array(b).shape

((128, 32, 32, 128), (128, 131072))

In [155]:
batches,row,col,d=conv0.shape
conv0.shape

(128, 64, 64, 128)

In [156]:
M=N=64
K=L=2

In [157]:
MK=NL=M//K
MK

32

In [212]:
def max_pool(self,inp,ksize=[2,2],stride=[2,2]):
    #inp[batches,row,col,d], kernels[ksz,ksz], stride[row,col]
    ksz=ksize[0]
    batches,row,col,d=inp.shape
    out_row,out_col=row//ksz,col//ksz
    ipp=inp.reshape(batches,out_row,ksz,out_col,ksz,d)
    output=ipp.max(axis=(2,4),keepdims=True)
    mask=((ipp-output)==0)
    #[batches,o_row,o_col,d]
    return output.squeeze(), mask

In [213]:
a,b=max_pool(1,conv0)

In [ ]:
def max_pool_back(self,errors,inp,max_index,ksize=[2,2],stride=[2,2]):
    #inp[batches,row,col,d], kernels[ksz,ksz], stride[row,col]
    ksz=ksize[0]
    batches,row,col,d=inp.shape
    out_row,out_col=row//ksz,col//ksz
    ipp=inp.reshape(batches,out_row,ksz,out_col,ksz,d)
    output=ipp.max(axis=(2,4),keepdims=True)
    mask=((ipp-output)==0)
    #[batches,o_row,o_col,d]
    return output.squeeze(), mask

In [158]:
ipp=conv0.reshape(batches,out_row,ksz,out_col,ksz,d)
output=ipp.max(axis=(2,4),keepdims=True)

In [181]:
ipp.shape,output.squeeze().shape

388 ns ± 3.27 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [186]:
%timeit gg=((ipp-output)==0)

155 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [211]:
gg.shape

(128, 32, 2, 32, 2, 128)

In [178]:
a.shape

(128, 32, 32, 128)

In [180]:
np.array_equal(a,output.squeeze())

True

In [34]:
c.shape

(128, 32, 32, 128)

In [79]:
tt=np.random.rand(3,4,4)

In [80]:
tt

array([[[0.25203656, 0.18987449, 0.03361191, 0.70183642],
        [0.19586411, 0.56849929, 0.08471081, 0.60763271],
        [0.58966047, 0.89792145, 0.79474758, 0.52592713],
        [0.31802628, 0.8764469 , 0.52431739, 0.66999025]],

       [[0.93599601, 0.81697813, 0.77262906, 0.84268487],
        [0.22837146, 0.49154422, 0.53719611, 0.35643433],
        [0.67304572, 0.33233793, 0.11068855, 0.92215175],
        [0.9602961 , 0.18935387, 0.70282529, 0.37144858]],

       [[0.88894938, 0.69638224, 0.37977611, 0.05904014],
        [0.55514087, 0.22413   , 0.15235257, 0.73005733],
        [0.90797748, 0.28522024, 0.22397752, 0.63607981],
        [0.79549962, 0.39947556, 0.29614264, 0.21173062]]])

In [189]:
ag=tt.reshape(3,2,2,2,2).max(axis=(2,4),keepdims=True)#.squeeze()

In [187]:
mask=(((tt.reshape(3,2,2,2,2)-tt.reshape(3,2,2,2,2).max(axis=(2,4),keepdims=True)).reshape(3,4,4))==0)

In [190]:
mask.shape,ag.shape

((3, 4, 4), (3, 2, 1, 2, 1))

In [201]:
%timeit (mask.reshape(3,2,2,2,2)*ag).reshape(3,4,4)

2.42 µs ± 31 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
